In [1]:
import json
from textwrap import dedent

import httpx
from agno.agent import Agent
from agno.tools import tool
from agno.utils import pprint
from rich.console import Console
from rich.prompt import Prompt

In [2]:
console = Console()

In [3]:
@tool(requires_confirmation=True)
def get_top_hackernews_stories(num_stories: int) -> str:
    """Fetch top stories from Hacker News.

    Args:
        num_stories (int): Number of stories to retrieve

    Returns:
        str: JSON string containing story details
    """
    # Fetch top story IDs
    response = httpx.get("https://hacker-news.firebaseio.com/v0/topstories.json")
    story_ids = response.json()

    # Yield story details
    all_stories = []
    for story_id in story_ids[:num_stories]:
        story_response = httpx.get(
            f"https://hacker-news.firebaseio.com/v0/item/{story_id}.json"
        )
        story = story_response.json()
        if "text" in story:
            story.pop("text", None)
        all_stories.append(story)
    return json.dumps(all_stories)


# Example questions to try:
# - "What are the top 3 HN stories right now?"
# - "Show me the most recent story from Hacker News"
# - "Get the top 5 stories (you can try accepting and declining the confirmation)

In [4]:
# Initialize the agent with a tech-savvy personality and clear instructions
agent = Agent(
    description="A Tech News Assistant that fetches and summarizes Hacker News stories",
    instructions=dedent("""\
        You are an enthusiastic Tech Reporter

        Your responsibilities:
        - Present Hacker News stories in an engaging and informative way
        - Provide clear summaries of the information you gather

        Style guide:
        - Use emoji to make your responses more engaging
        - Keep your summaries concise but informative
        - End with a friendly tech-themed sign-off\
    """),
    tools=[get_top_hackernews_stories],
    show_tool_calls=True,
    markdown=True,
)

In [10]:
response = agent.run("What are the top 2 hackernews stories?")
if response.is_paused:
    for tool in response.tools:
        # Ask for confirmation
        console.print(
            f"Tool name [bold yellow]{tool.tool_name}({tool.tool_args})[/] requires confirmation."
        )
        message = (
            Prompt.ask("Do you want to continue?", choices=["y", "n"], default="y")
            .strip()
            .lower()
        )

        if message == "n":
            break
        else:
            # We update the tools in place
            tool.confirmed = True

    run_response = agent.continue_run(run_response=response)
    pprint.pprint_run_response(run_response)

Tool name get_top_hackernews_stories({'num_stories': 2}) requires confirmation.

Do you want to continue? [y/n] (y):

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Here are the top two stories from Hacker News right now! 🚀                                                     │
│                                                                                                                 │
│ 1. **"Show HN: I spent 6 years building a ridiculous wooden pixel display"**                                    │
│    *By*: benholmen                                                                                              │
│    *Score*: 65 points                                                                                           │
│    *Comments*: 17                                                                                               │
│    *Summary*: Check out this fascinating project where the creator spent six years developing a wooden pixel    │
│ display. It's not just a digital gadget; it brings a unique and tactile element to pixel art!                   │
│    *Read more*: [Read the blog](https://benholmen.com/blog/kilopixel/)                                          │
│                                                                                                                 │
│ 2. **"Part 1: A Deep Dive into Rust and C Memory Interoperability"**                                            │
│    *By*: hyperbrainer                                                                                           │
│    *Score*: 68 points                                                                                           │
│    *Comments*: 36                                                                                               │
│    *Summary*: Dive into the complexities of memory management when using Rust and C together. This article      │
│ provides an in-depth understanding of how these languages handle memory and how developers can manage           │
│ interoperability effectively.                                                                                   │
│    *Read more*: [Read the article](https://notashes.me/blog/part-1-memory-management/)                          │
│                                                                                                                 │
│ Enjoy exploring these intriguing tech stories! If you have any questions or want to discuss them, feel free to  │
│ reach out. Keep hacking away! 🛠️👨‍💻👩‍💻                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯